In [ ]:
import pandas as pd
from datetime import datetime
import re
import pandas as pd
from datetime import datetime
import re
import googlemaps


class AEDPoint:
    def __init__(self, id, x, y, address, indication, schedule, taken):
        self.id = id
        self.x = x
        self.y = y
        self.address = address
        self.indication = indication
        self.schedule = schedule
        self.taken = taken

class MapAEDS:
    def __init__(self):
        self.points = {}

    def add_point(self, point, availability):
        self.points[point.id] = [point, availability]
        print("Succesfully Added")

    def delete_point(self, point):
        if point in self.points:
            del self.points[point.id]
            print(f"Point with ID {point.id} deleted.")
        else:
            print("Point ID not found.")

    def update_availability(self, id, method, sit = None, time = None):
        
        if method == 1:
            if is_in_schedule(self.points[id][0].schedule, time) == True:
                self.points[id][1] = True
            else: 
                self.points[id][1] = False
        if method == 2: 
            if sit == 'take':
                self.points[id][1] = False
                self.points[id][0].taken = True 
            elif sit == 'return':
                self.points[id][1] = True
                self.points[id][0].taken = False 
                            

    def points_within_radius(self, user_loc, radius):
        # Initialize an empty list to store the points within the circle
        points_within_circle = {}
        while radius > 0:
            points_within_circle = {}
            found_points = 0  # Track the number of points found within the current radius
            for point in self.points:
                if self.points[point][1] == True:
                    # Get the longitude and latitude from the row
                    lat, lon = self.points[point][0].x, self.points[point][0].y
                    if lon < -90 or lon > 90 or lat < -90 or lat > 90:
                        continue
                    # Calculate the distance from the point to the center of the circle
                    distance = geodesic((lat, lon), user_loc).km
                    # If the distance is less than or equal to the radius, add the point to the list
                    if distance <= radius:
                        points_within_circle[point] = (lat, lon)
                        print(points_within_circle)
                        found_points += 1
                        if found_points == 24:
                            break
            if 0 < found_points <= 23:
                break  # If no points were found within the current radius, exit the loop
            elif found_points == 0:
                radius += 5 # Decrease radius
            else:
                radius -= 0.5
            print(points_within_circle)

        return points_within_circle

all_aeds = pd.read_excel("C:/Users/jorge/Downloads/aed_all_inv.xlsx")
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]
start_index = 15227

# Select the last 1000 rows starting from the defined index until the end and swap latitude and longitude columns within that subset
last_1000_rows = all_aeds.iloc[start_index:]
last_1000_rows['latitude'], last_1000_rows['longitude'] = last_1000_rows['longitude'], last_1000_rows['latitude']

# Update the last 1000 rows in the original DataFrame
all_aeds.iloc[start_index:] = last_1000_rows

all_aeds = all_aeds.dropna(subset=['longitude'])

# Reset index
all_aeds = all_aeds.reset_index(drop=True)

for i in range(0, len(all_aeds)):
    for column in weekdays:
        if type(all_aeds[column][i]) == str:
            if all_aeds[column][i][-1] != ']':
                new_val = all_aeds[column][i].strip()
                all_aeds[column][i] = new_valweekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]

# Example usage:
maps = MapAEDS()
for i in range(0, len(all_aeds)):
    dict_schedule = {}
    for weekday in weekdays:
        if isinstance(all_aeds[weekday][i] , str) or all_aeds[weekday][i] == 0:
            dict_schedule[weekday] = all_aeds[weekday][i]            
    point = AEDPoint(i, all_aeds['latitude'][i], all_aeds['longitude'][i], all_aeds['address'][i], all_aeds['location'][i], dict_schedule, False)
    maps.add_point(point, True)
    
current_time = datetime.now()

for point in maps.points:
    if maps.points[point][0].taken == False:
        maps.update_availability(point, 1, time = current_time)

nearby_points = maps.points_within_radius(user_loc, 10.8)